In [1]:
import joblib
import sys
sys.path.insert(0, r'D:/downoloads/FraudDetection/FraudDetection-master')
from nic_optimization import (
    genetic_algorithm,
    particle_swarm_optimization,
    artificial_immune_system
)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    classification_report,
    confusion_matrix
)

In [ ]:
# Load the saved models and data
models = joblib.load('models.joblib')
dataset = joblib.load('train_test_data.joblib')
X_train, X_test = dataset['X_train'], dataset['X_test']
y_train, y_test = dataset['y_train'], dataset['y_test']


In [3]:
# --- RandomForest ---
def fitness_rf(params):
    n_estimators, max_depth = params
    model = RandomForestClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        class_weight={0:1, 1:(1 - y_train.mean())/y_train.mean()},
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_prob)

def evaluate_rf(params):
    n_estimators, max_depth = params
    model = RandomForestClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        class_weight={0:1, 1:(1 - y_train.mean())/y_train.mean()},
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    return {
        'roc_auc'           : roc_auc_score(y_test, y_prob),
        'avg_precision'     : average_precision_score(y_test, y_prob),
        'confusion_matrix'  : confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred)
    }

In [4]:
# --- GradientBoosting ---
def fitness_gb(params):
    learning_rate, n_estimators = params
    model = GradientBoostingClassifier(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        random_state=42
    )
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_prob)

def evaluate_gb(params):
    learning_rate, n_estimators = params
    model = GradientBoostingClassifier(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    return {
        'roc_auc'           : roc_auc_score(y_test, y_prob),
        'avg_precision'     : average_precision_score(y_test, y_prob),
        'confusion_matrix'  : confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred)
    }

In [5]:
# --- SVM ---
#def fitness_svm(params):
#    C, gamma = params
#    model = SVC(kernel='rbf', C=C, gamma=gamma,
#                probability=True, random_state=42)
#    model.fit(X_train, y_train)
#    y_prob = model.predict_proba(X_test)[:, 1]
#    return roc_auc_score(y_test, y_prob)

#def evaluate_svm(params):
#    C, gamma = params
#    model = SVC(kernel='rbf', C=C, gamma=gamma,
#                probability=True, random_state=42)
#    model.fit(X_train, y_train)
#    y_pred = model.predict(X_test)
#    y_prob = model.predict_proba(X_test)[:, 1]
#    return {
#        'roc_auc'           : roc_auc_score(y_test, y_prob),
#        'avg_precision'     : average_precision_score(y_test, y_prob),
#        'confusion_matrix'  : confusion_matrix(y_test, y_pred),
#        'classification_report': classification_report(y_test, y_pred)
#    }

In [ ]:
# Matching models and functions
fitness_map = {
    #'SVC'               : (fitness_svm, evaluate_svm),
    'RandomForest'      : (fitness_rf,  evaluate_rf),
    'GradientBoosting'  : (fitness_gb,  evaluate_gb)
}

In [ ]:
algorithms = {
    'GeneticAlgorithm'          : genetic_algorithm,
    'ParticleSwarmOptimization' : particle_swarm_optimization,
    'ArtificialImmuneSystem'    : artificial_immune_system
}

In [ ]:
if __name__ == '__main__':
    for model_name, (fit_fn, eval_fn) in fitness_map.items():
        print(f"\n=== Optimisation of the model: {model_name} ===")
        for algo_name, algo_fn in algorithms.items():
            print(f"\n--- Algorithm: {algo_name} ---")
            best_params, history = algo_fn(fit_fn)
            metrics = eval_fn(best_params)
            print("Metrics:")
            for key, val in metrics.items():
                print(f"{key}:\n{val}\n")




=== Optimisation of the model: RandomForest ===

--- Algorithm: GeneticAlgorithm ---
GA early stop at gen 6, no improvement for 3 gens
Metrics:
roc_auc:
0.9575585490977103

avg_precision:
0.8045401881719753

confusion_matrix:
[[170883     80]
 [  2479   3720]]

classification_report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    170963
           1       0.98      0.60      0.74      6199

    accuracy                           0.99    177162
   macro avg       0.98      0.80      0.87    177162
weighted avg       0.99      0.99      0.99    177162



--- Algorithm: ParticleSwarmOptimization ---
PSO early stop at iter 3, no improvement for 3 iters
Metrics:
roc_auc:
0.9380401341717653

avg_precision:
0.7520401886476354

confusion_matrix:
[[170843   120]
 [  3000   3199]]

classification_report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    170963
           1       0.96      0